In [17]:
import numpy as np
import pandas as pd
import scipy.stats as si
from opt_assistant import Opt_Assistant

opt_info = Opt_Assistant()
from stock_centre import *

hq = Stock_Data_Centre()
ts_obj = TuShare()
pro = ts_obj.pro
from param_standard import *

code = '10000001.XSHG'
date = '2015-03-02'

underlying_symbol = '510050'
start_date = '2021-07-01'
end_date = '2023-06-30'


def calculate_greeks(code, date, interest_rate_type='1y', his_vol_len=20):
    '''
    计算一张期权的希腊字母（已乘乘数）
    :param code: 期权代码
    :param date: 需要求希腊值的日期
    :param interest_rate_type: Shibor的类型，范围：1w, 2w, 1m, 3m, 6m, 9m, 1y
    :param his_vol_len: int, 历史波动率的时间跨度（天）
    :return: 理论价格，delta，gamma，vega，theta，rho, 杠杆率
    '''

    global underlying_symbol

    period_end_date = pd.to_datetime(date, format='%Y-%m-%d')

    period_start_date = period_end_date.replace(year=period_end_date.year - 1)

    contract_info = opt_info.get_contract_info(code=code)
    
    daily_info = opt_info.get_daily_info(code=code, date=date)

    underlying_symbol_for_stock = param_standard(underlying_symbol=underlying_symbol)['underlying_symbol_for_stock']
    #  标的历史价格
    if contract_info['underlying_type'].iloc[0] == 'INDEX':
        underlying_prc_his = hq.get_hq(code=underlying_symbol_for_stock, start_date=period_start_date,
                                       end_date=period_end_date,
                                       index_data=True)
    elif contract_info['underlying_type'].iloc[0] == 'ETF':
        underlying_prc_his_Tushare = pro.fund_daily(ts_code=underlying_symbol_for_stock,
                                                    start_date=period_start_date.strftime('%Y%m%d'),
                                                    end_date=period_end_date.strftime('%Y%m%d'))
        underlying_prc_his = underlying_prc_his_Tushare.set_index(
            pd.to_datetime(underlying_prc_his_Tushare['trade_date'], format='%Y%m%d').dt.strftime('%Y-%m-%d'))
        underlying_prc_his.sort_index(ascending=False, inplace=True)
    
    contract_unit = contract_info['contract_unit'].iloc[0]
    
    s = underlying_prc_his['close'].iloc[0]

    k = contract_info['exercise_price'].iloc[0]

    r = pro.shibor(start_date=period_end_date.strftime('%Y%m%d'), end_date=period_end_date.strftime('%Y%m%d'))[
            interest_rate_type].iloc[0] / 100

    exercise_date = contract_info['exercise_date'].iloc[0]

    t = len(hq.get_trade_date(start_date=period_end_date, end_date=exercise_date, only_list=True)) / 365

    sigma = underlying_prc_his.head(his_vol_len)['close'].pct_change().std() * np.sqrt(252)

    if contract_info['contract_type'].iloc[0] == 'CO':
        n = 1
    elif contract_info['contract_type'].iloc[0] == 'PO':
        n = -1

    d1 = (np.log(s / k) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)

    theo_price = round(contract_unit * n * (s * si.norm.cdf(n * d1) - k * np.exp(-r * t) * si.norm.cdf(n * d2)), 2)

    delta = round(contract_unit * n * si.norm.cdf(n * d1), 2)

    gamma = round(contract_unit * si.norm.pdf(d1) / (s * sigma * np.sqrt(t)), 2)

    vega = round(contract_unit * (s * si.norm.pdf(d1) * np.sqrt(t)) / 100, 2)

    theta = round(contract_unit * (-1 * (s * si.norm.pdf(d1) * sigma) / (2 * np.sqrt(t)) - n * r * k * np.exp(-r * t) * si.norm.cdf(n * d2)) / 365, 2)

    rho = round(contract_unit * n * (t * k * np.exp(-r * t) * si.norm.cdf(d2) * 0.01), 2)

    leverage = round(s * delta / daily_info['settle_price'].iloc[0], 2)

    return {'theo_price': theo_price, 'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta, 'rho': rho,
            'leverage': leverage}


print(calculate_greeks(code, date, interest_rate_type='1y', his_vol_len=20))


{'theo_price': 2478.8, 'delta': 9695.4, 'gamma': 4899.79, 'vega': 3.74, 'theta': -5.47, 'rho': 10.45, 'leverage': 94476.93}
